# HOMEWORK 03

## Exercise 1


1) Draw the Bayesian Network representing the joint distribution: 

$P(A, B, C, D, E, F, G, H) = P(A)\cdot P(B|A)\cdot P(C)\cdot P(D|B)\cdot P(E)\cdot P(F|A)\cdot P(G|D, F)\cdot P(H|E, B)$

![](image_1.jpg)

1) Indicate whether the following statements on conditional independence are True or False and motivate your answer.

First of all, we need to remind when two r.v. are conditional independent, given a third one.

__Definition of conditional independence__

Let be $a$, $b$ and $c$ random variables, so $a$ is conditional independence of $b$ given $c$ ($a \perp \!\!\! \perp b | c$) iff:

- $P(a | b, c) = P(a | c)$, or
- $P(a, b | c) = P(a | c) \cdot P(b | c)$.

Another possible way to define the conditional independence is by checking directly the bayesian network. So, let be $A$, $B$ and $C$ disjoint subset of the bayesian network, if all paths from a node in $A$ to a node in $B$ are blocked by a node in $C$, then $A \perp \!\!\! \perp B | C$. In particular a path from a node $a$ in $A$ to a node $b$ in $B$ is blocked by a node $c$ in $C$ iff:

- The node $c$ is obsorved and the path is "head to tail" (with $c$ the middle node of the triple), or
- The node $c$ is not obsorved neither any descendent of $c$ (i.e. neither $c$ neither $c$-descendents are part of the $C$ subset), and the path is "head to head" in $c$.

Where a path is an undirected link between a node $a$ and a node $b$, passing through no nodes or several nodes.

a) $A \perp \!\!\! \perp B$

There are two paths that connect $A$ and $B$, the first one is $A \rightarrow B$ and it's no blocked, while the second one is $A \rightarrow F \rightarrow G \rightarrow D \rightarrow B$ that is blocked in $G$. So, $A \perp \!\!\! \perp B$ is false.

b) $A \perp \!\!\! \perp C$

There are no paths connecting $A$ and $C$, so $A \perp \!\!\! \perp C$ is true.

c) $A \perp \!\!\! \perp D |{B, H}$

There are two paths that connect $A$ and $D$, the first one is $A \rightarrow F \rightarrow G \rightarrow D$ and it's blocked in $G$, while the second one is $A \rightarrow B \rightarrow D$ that is blocked in $B$. So, $A \perp \!\!\! \perp D |{B, H}$ is true.

d) $A \perp \!\!\! \perp E | F$

There are two paths that connect $A$ and $E$, the first one is $A \rightarrow B \rightarrow H \rightarrow F$ and it's blocked in $H$, while the second one is $A \rightarrow F \rightarrow G \rightarrow D \rightarrow B \rightarrow H \rightarrow E$ that is blocked in $F$. So, $A \perp \!\!\! \perp D |{B, H}$ is true.

e) $G \perp \!\!\! \perp E | B$

There are two paths that connect $G$ and $E$, the first one is $G \rightarrow D \rightarrow B \rightarrow H \rightarrow E$ and it's blocked in $B$, while the second one is $G \rightarrow F \rightarrow A \rightarrow B \rightarrow H \rightarrow E$ that is blocked in $B$. So, $G \perp \!\!\! \perp E | B$ is true.

f) $F \perp \!\!\! \perp C | D$

There are no paths connecting $F$ and $C$, so $F \perp \!\!\! \perp C | D$ is true.

g) $E \perp \!\!\! \perp D | B$

There are two paths that connect $E$ and $D$, the first one is $E \rightarrow H \rightarrow B \rightarrow D$ and it's blocked in $H$, while the second one is $E \rightarrow H \rightarrow B \rightarrow A \rightarrow F \rightarrow G \rightarrow D$ that is blocked in $H$. So, $E \perp \!\!\! \perp D | B$ is true.

h) $C \perp \!\!\! \perp H | G$

There are no paths connecting $H$ and $C$, so $C \perp \!\!\! \perp H | G$ is true.

**Note: the "$\rightarrow$" symbol has the only scope to show the possible path and doesn't consider the network direction.**

## Exercise 2

a) Build the generative model corresponding to the directed graph

![](image_2.png)

__The generative model__

- $\theta$ ~ Dirichlet$_{K}(\alpha)$;
- $\mu_{K}$ ~ $N(0, \eta^{2})$ for the mixture components;
- for each point $t+j$, where $j = -1, 0, 1$:
    - if $j = -1$ then $z_{t+j} | \theta$ ~ $Categorical(\theta)$;
    - otherwise $z_{t+j} | \theta, z_{t+j-1}$ ~ $Categorical(\theta_{z_{t+j-1}})$;
    - for each $j$, $x_{t+j} | z_{t+j}, \mu_{z_{t+j}}$ ~ $N(\mu_{z_{t+j}}, 1)$.

The joint distribution factorizes as:

$p(\theta, \mu, z, x | \alpha, \eta) = 
\prod_{i = 0}^{K} \left \{ p(\theta_{i} | \alpha) p(\mu_{i} | \eta) p(z_{t-1} | \theta_{i}) p(x_{t-1} | \mu_{i}, z_{t-1}) \left [ \prod_{j = -1}^{N} p(z_{t+j} | \theta_{i}, z_{t+j-1}) p(x_{t+j} | \mu_{i}, z_{t+j}) \right ] \right \}$

From which we can define the posterior distribution as:

$p(\theta, \mu, z | x, \eta, \alpha) = \frac{p(\theta, \mu, z, x | \alpha, \eta)}{p(x | \alpha, \eta)}$

b) Using Dirichlet, Categorical and Normal distributions and supposing that $K = 2$. Then, write a pyro implementation of the resulting model.

In [2]:
import pyro
import torch
import pyro.distributions as dist


# number of components
K = 2 

# hyperparameters
alpha = 0.5
sigma_0 = 10.0

def model(data):
    
    N = len(data)
    
    # global variables
    with pyro.plate('hyperparameters', K):
        theta = pyro.sample('theta', dist.Dirichlet(alpha * torch.ones(K)))

    
    with pyro.plate('components', K):
        mu = pyro.sample('mu', dist.Normal(0., sigma_0))
    
    z = list()
    
    for i in range(N):
        if i == 0: # first case z_0
            z_f = pyro.sample('z', dist.Categorical(probs = theta[0])) # theta is fixed for the first one z
            z.append(z_f)
        else: # all others cases
            z_r = pyro.sample('z', dist.Categorical(probs = theta[z_f]))
            z.append(z_r)
            z_f = z_r
        x = pyro.sample('x', dist.Normal(mu[z_f], 1), obs = data)
    
    z = torch.stack(tuple(z),0)
    print("theta =",theta,"\nmu =",mu,"\nz =", z,"\nx =",  x)
    
model(data = [7, 0.8, 0.1])

theta = tensor([[0.8686, 0.1314],
        [0.8617, 0.1383]]) 
mu = tensor([-2.1580, 12.7415]) 
z = tensor([0, 0, 1]) 
x = [7, 0.8, 0.1]
